## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


# callback
from tensorflow.keras.callbacks import ModelCheckpoint

# Mount Drive for Colab
# https://stackoverflow.com/a/76415665/22212330

from google.colab import drive
drive.mount('/content/drive')

path = r'/content/drive/Othercomputers/My PC/Data Analytics Bootcamp/21-Neural-Networks-Deep-Learning/0_Challenge/deep-learning-challenge'

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Target variable="IS_SUCCESSFUL"

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_bin_counts = application_df['APPLICATION_TYPE'].value_counts()
app_bin_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace =  list(app_bin_counts[app_bin_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_bin_counts = application_df['CLASSIFICATION'].value_counts()
class_bin_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_bin_counts[class_bin_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace =  list(class_bin_counts[class_bin_counts < 2000].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_numeric = pd.get_dummies(application_df)


In [11]:
application_df_numeric.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays
X = application_df_numeric.drop('IS_SUCCESSFUL', axis=1)
y = application_df_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_of_features = len(X_train_scaled[0])#
nodes_l1 = 80
nodes_l2 = 30
nodes_l3 = 20
nodes_l4 = 10
print(f'number of input features = {number_of_features}')


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_l1, input_dim=number_of_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_l2, activation="relu"))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_l3, activation="sigmoid"))


# third hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_l4, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

number of input features = 38
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3120      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 20)                620       
                                                                 
 dense_3 (Dense)             (None, 10)                210       
                                                                 
 dense_4 (Dense)             (None, 1)                 11        
                                                                 
Total params: 6391 (24.96 KB)
Trainable params: 6391 (24.96 KB)
Non-trainable params: 0 (0.00 Byte)
__________________________________________________________

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [16]:
# Create a callback that saves the model's weights every five epochs.

checkpoint_filepath = path + '/2_Model_Optimise_2/weights_for_epoch_no_{epoch:02d}.hdf5'
print(checkpoint_filepath)

# For callbacks, use deprecated period=5 to achieve checkpoint every 5 epochs; see: https://stackoverflow.com/a/73018801

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    # save_freq='epoch',
    period=5,
    verbose = 1)



/content/drive/Othercomputers/My PC/Data Analytics Bootcamp/21-Neural-Networks-Deep-Learning/0_Challenge/deep-learning-challenge/2_Model_Optimise_2/weights_for_epoch_no_{epoch:02d}.hdf5


In [17]:
# Train the model

# https://www.tensorflow.org/tutorials/keras/save_and_load
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[model_checkpoint_callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5768 - accuracy: 0.7198
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5623 - accuracy: 0.7254
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5586 - accuracy: 0.7278
Epoch 5/100
802/804 [============================>.] - ETA: 0s - loss: 0.5557 - accuracy: 0.7306
Epoch 5: saving model to /content/drive/Othercomputers/My PC/Data Analytics Bootcamp/21-Neural-Networks-Deep-Learning/0_Challenge/deep-learning-challenge/2_Model_Optimise_2/weights_for_epoch_no_05.hdf5
804/804 [==============================] - 6s 7ms/step - loss: 0.5561 - accuracy: 0.7303
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5564 - accuracy: 0.7301
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5546 - accuracy: 0.7322
Epoch 

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5662 - accuracy: 0.7245 - 508ms/epoch - 2ms/step
Loss: 0.5661934614181519, Accuracy: 0.7245481014251709


In [19]:
# Export our model to HDF5 file
savepathname = path + '/2_Model_Optimise_2/AlphabetSoupCharity.h5'
nn.save(savepathname)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
